# Content
1. Importing Libraries and Data
2. Data checks ans preparation
3. PII issues
4. Create flags for regions
5. Spending and ordering habits across regions
6. Creating dataframe to show aggregate values for regions
7. Excluding Low freguency customers
8. Export

### Import Libraries 

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

### Import Data

In [2]:
# define path

path = r'C:\Users\13526\Documents\Instacart Basket Analysis'

In [3]:
df = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'full_instacart_data.pkl'))

### Data Checks

In [4]:
df.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,customer_id,order_number,orders_day_of_week,order_hour_of_day,...,order_frequency_flag,first_name,surname,gender,state,age,date_joined,#_of_dependants,marital_status,income
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,Frequent customer,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,49620
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,Frequent customer,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,49620
2,907,Premium Sliced Bacon,106,12,20.0,3160996,138,1,5,13,...,Frequent customer,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,49620
3,907,Premium Sliced Bacon,106,12,20.0,2254091,138,10,5,14,...,Frequent customer,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,49620
4,1000,Apricots,18,10,12.9,505689,138,9,6,12,...,Frequent customer,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,49620


In [5]:
df.shape

(32406041, 32)

## Consider Security Implications

For security purposes, there is no need to have customer names. Customer_id should be sufficient.

In [8]:
# Drop the first and last name columns. The columns _merge and aisle_id can also be dropped while at it

df2 = df.drop(columns=['first_name', 'surname', 'aisle_id','_merge'])

In [10]:
# Check to make sure successfully dropped

df2.head()

,product_id,product_name,department_id,prices,order_id,customer_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,...,spending_flag,median_order_frequency,order_frequency_flag,gender,state,age,date_joined,#_of_dependants,marital_status,income
0,1,Chocolate Sandwich Cookies,19,5.8,3139998,138,28,6,11,3.000000,...,Low spender,8.0,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620
1,1,Chocolate Sandwich Cookies,19,5.8,1977647,138,30,6,17,20.000000,...,Low spender,8.0,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620
2,907,Premium Sliced Bacon,12,20.0,3160996,138,1,5,13,1.110352,...,Low spender,8.0,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620
3,907,Premium Sliced Bacon,12,20.0,2254091,138,10,5,14,6.000000,...,Low spender,8.0,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620
4,1000,Apricots,10,12.9,505689,138,9,6,12,7.000000,...,Low spender,8.0,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620


## Create flags for regions

In [11]:
# create series for each region with all appropriate states
Northeast = ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']
Midwest = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']
South = ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']
West = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona',  'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']

In [13]:
# create a new region column identifying

df2.loc[df2['state'].isin(Northeast), 'Region'] = 'Northeast'

In [14]:
df2.loc[df2['state'].isin(Midwest), 'Region'] = 'Midwest'

In [16]:
df2.loc[df2['state'].isin(South), 'Region'] = 'South'

In [17]:
df2.loc[df2['state'].isin(West), 'Region'] = 'West'

In [18]:
# make sure all records were allocated to a region

df2['Region'].value_counts(dropna=False)

South        10792300
West          8293217
Midwest       7597585
Northeast     5722939
Name: Region, dtype: int64

All records were seccessfully allocated to a region

## Spending and ordering habits across regions

In [21]:
# Group by region and aggregate by customer id in order to see which region has more customers

df2[['Region', 'customer_id']].groupby('Region').agg({'customer_id': pd.Series.nunique,}).sort_values(by='customer_id', ascending=False)

,customer_id
Region,
South,68737
West,52565
Midwest,48519
Northeast,36388


The South has the most customers and the Northeast has the least

In [23]:
# Important to note that the South also has the most states alloted to it's region

df2[['Region', 'state']].groupby('Region').agg({'state': pd.Series.nunique,}).sort_values(by='state', ascending=False)

,state
Region,
South,17
West,13
Midwest,12
Northeast,9


The South also has the most states and the Northeast has the least

In [25]:
# Check to see which region has sold the most products

df2[['Region', 'product_id']].groupby(['Region']).agg('count').sort_values(by='product_id', ascending=False)

,product_id
Region,
South,10792300
West,8293217
Midwest,7597585
Northeast,5722939


Again we see that the South has sold the most products and the Northeast has sold the least

In [26]:
# checking what region customers spend the most in sales

df2[['prices', 'Region']].groupby('Region').agg('sum').sort_values(by ='prices', ascending=False)

,prices
Region,
South,131616128.0
Midwest,96605424.0
West,93876472.0
Northeast,66127652.0


Southern customers were the top total spenders and Northeastern customers were the least

In [28]:
# Use crosstab to see the frequency of low and high spending customers in each region

cross = pd.crosstab(df2['Region'], df2['spending_flag'], dropna=False).sort_index()

In [29]:
cross

spending_flag,High spender,Low spender
Region,,
Midwest,155979,7441606
Northeast,108226,5614713
South,209693,10582607
West,160354,8132863


The South has the highest frequency of High spending customers. The Northeast has the least

In [31]:
# Use crosstab to see the customer frequency per region

cross2 = pd.crosstab(df2['Region'], df2['order_frequency_flag'], dropna=False).sort_index()

In [32]:
cross2

order_frequency_flag,Frequent customer,Non-frequent customer,Regular customer
Region,,,
Midwest,5305686,640112,1651787
Northeast,4023524,469957,1229458
South,7602456,898007,2291837
West,5864364,678758,1750095


The South also has the most "frequent customers". The Northeast has the least "frequent customers"

In [33]:
# Use crosstab to see if there are more loyal customers in different regions

cross3 = pd.crosstab(df2['Region'], df2['loyalty_flag'], dropna=False).sort_index()

In [34]:
cross3

loyalty_flag,Loyal Customer,New customer,Regular customer
Region,,,
Midwest,2373854,1472626,3751105
Northeast,1841833,1100255,2780851
South,3405949,2074492,5311859
West,2662760,1596877,4033580


The South has the most loyal customers, and the Northeast has the least

## Create dataframe to show aggregate values for regions

In [37]:
# create a new df that summarizes data by region
# rename the columns so they are clear

df_region = df2[['prices', 'Region', 'customer_id', 'order_id']].groupby(['Region']).agg({'prices':'sum', 'customer_id': pd.Series.nunique, 'order_id': pd.Series.nunique})

In [44]:
df_region = df_region.rename(columns={'prices':'Prices_sum', 'customer_id': 'User_count', 'order_id': 'Order_count'})

In [45]:
df_region

,Prices_sum,User_count,Order_count
Region,,,
Midwest,96605424.0,48519,754275
Northeast,66127652.0,36388,567162
South,131616128.0,68737,1071562
West,93876472.0,52565,821686


In [46]:
# create calculated columns based on the aggregated data

df_region['price_per_order'] = df_region['Prices_sum']/df_region['Order_count']
df_region['price_per_user'] = df_region['Prices_sum']/df_region['User_count']
df_region['order_per_user'] = df_region['Order_count']/df_region['User_count']

In [48]:
df_region.sort_values(by='price_per_user', ascending=False)

,Prices_sum,User_count,Order_count,price_per_order,price_per_user,order_per_user
Region,,,,,,
Midwest,96605424.0,48519,754275,128.077192,1991.084400,15.545972
South,131616128.0,68737,1071562,122.826423,1914.778474,15.589304
Northeast,66127652.0,36388,567162,116.593940,1817.292844,15.586512
West,93876472.0,52565,821686,114.248596,1785.912147,15.631808


## Excluding low frequency customers

In [50]:
# Create variable for # of orders made by each user.The count of user_id would be the number of orders that user has made

df2['Total_orders']=df2.groupby(['customer_id']).customer_id.transform('count')

In [51]:
# Create exclusion flag by counting the amount of times Customer_id shows up in dataframe, less than 5 is low activity

results_2=[]
for total in df2['Total_orders']:
    if total < 5:
        results_2.append('Low Activity')
    else:
        results_2.append('Regular Activity')

In [52]:
# Include column in dataframe

df2['Activity_flag']=results_2

In [53]:
# Check how many low activity customers there are

df2['Activity_flag'].value_counts(dropna=False)

Regular Activity    32403157
Low Activity            2884
Name: Activity_flag, dtype: int64

In [54]:
# Create subset of low activity customers

low_activity_customers= df2[df2['Activity_flag']=='Low Activity']

In [55]:
# Check subset to make sure successful

low_activity_customers.shape

(2884, 31)

In [56]:
# Export low activity subset

low_activity_customers.to_csv(os.path.join(path, '02 Data','Prepared Data', 'low_activity_customers.csv'))

In [57]:
# Exclude sample from dataset

df2.drop(df2[df2['Activity_flag']=='Low Activity'].index, inplace=True)

In [58]:
# Check to make sure successful

df2['Activity_flag'].value_counts()

Regular Activity    32403157
Name: Activity_flag, dtype: int64

In [59]:
# Viewing dataset
pd.set_option('display.max_columns', None)
df2.head()

,product_id,product_name,department_id,prices,order_id,customer_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,busiest_day,Busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price,spending_flag,median_order_frequency,order_frequency_flag,gender,state,age,date_joined,#_of_dependants,marital_status,income,Region,Total_orders,Activity_flag
0,1,Chocolate Sandwich Cookies,19,5.8,3139998,138,28,6,11,3.000000,5,0,Regularly busy,Regulary busy days,Most orders,32,Regular customer,6.935811,Low spender,8.0,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest,148,Regular Activity
1,1,Chocolate Sandwich Cookies,19,5.8,1977647,138,30,6,17,20.000000,1,1,Regularly busy,Regulary busy days,Most orders,32,Regular customer,6.935811,Low spender,8.0,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest,148,Regular Activity
2,907,Premium Sliced Bacon,12,20.0,3160996,138,1,5,13,1.110352,4,0,Regularly busy,Regulary busy days,Most orders,32,Regular customer,6.935811,Low spender,8.0,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest,148,Regular Activity
3,907,Premium Sliced Bacon,12,20.0,2254091,138,10,5,14,6.000000,1,1,Regularly busy,Regulary busy days,Most orders,32,Regular customer,6.935811,Low spender,8.0,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest,148,Regular Activity
4,1000,Apricots,10,12.9,505689,138,9,6,12,7.000000,5,0,Regularly busy,Regulary busy days,Most orders,32,Regular customer,6.935811,Low spender,8.0,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest,148,Regular Activity


In [60]:
# Create subset of regular activity customers
df3= df2[df2['Activity_flag']=='Regular Activity']

In [62]:
# Export regular activity subset
df3.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'reg_activity_customers.pkl'))